### Fine-tuning on T5 model

In [ ]:
import torch
from transformers import T5Tokenizer, T5ForConditionalGeneration, Seq2SeqTrainingArguments, Seq2SeqTrainer
from datasets import load_dataset
import numpy as np

# 1. Load Dataset and Tokenizer
dataset = load_dataset("dair-ai/emotion")
tokenizer = T5Tokenizer.from_pretrained("t5-base")

import os
import torch
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments


os.environ["WANDB_DISABLED"] = "true"
os.environ["NCCL_P2P_DISABLE"] = "1"
os.environ["NCCL_IB_DISABLE"] = "1"
os.environ["NCCL_DEBUG"] = "INFO"

# Check if GPU is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# 2. Preprocess Data for T5
def preprocess_function(examples):
    inputs = ["emotion classification: " + text for text in examples["text"]]
    labels = [["joy", "sadness", "anger", "fear", "surprise","disgust"][label] for label in examples["label"]]
    
    model_inputs = tokenizer(
        inputs,
        max_length=128,
        padding="max_length",
        truncation=True,
        return_tensors="pt"
    )
    
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(
            labels,
            max_length=8,
            padding="max_length",
            truncation=True,
            return_tensors="pt"
        )
    
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

tokenized_datasets = dataset.map(preprocess_function, batched=True)

# 3. Load T5-base Model
model = T5ForConditionalGeneration.from_pretrained("t5-base").to("cuda" if torch.cuda.is_available() else "cpu")

# 4. Training Configuration
training_args = Seq2SeqTrainingArguments(
    output_dir="./t5-emotion",
    evaluation_strategy="epoch",
    learning_rate=3e-4,
    per_device_train_batch_size=64,
    per_device_eval_batch_size=32,
    num_train_epochs=100,
    weight_decay=0.01,
    predict_with_generate=True,
    logging_dir="./logs",
    report_to="none",
    save_strategy="epoch",
)

# 5. Create Trainer
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
)

# 6. Train and Save
trainer.train()
model.save_pretrained("models/")
tokenizer.save_pretrained("models/")


/opt/conda/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-03-25 11:59:37.777534: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-03-25 11:59:37.789226: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1742903977.803695  888827 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1742903977.808039  888827 cuda_blas.cc:1418] Unable to register 

Using device: cuda


Map:   0%|          | 0/16000 [00:00<?, ? examples/s]/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:3970: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(
Map: 100%|██████████| 2000/2000 [00:00<00:00, 2874.16 examples/s]
/opt/conda/lib/python3.10/site-packages/transformers/training_args.py:1594: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


02aa490fc410:888827:888827 [0] NCCL INFO cudaDriverVersion 12020
02aa490fc410:888827:888827 [0] NCCL INFO Bootstrap : Using eth0:172.17.0.2<0>
02aa490fc410:888827:888827 [0] NCCL INFO NET/Plugin: No plugin found (libnccl-net.so)
02aa490fc410:888827:888827 [0] NCCL INFO NET/Plugin: Plugin load returned 2 : libnccl-net.so: cannot open shared object file: No such file or directory : when loading libnccl-net.so
02aa490fc410:888827:888827 [0] NCCL INFO NET/Plugin: Using internal network plugin.
NCCL version 2.21.5+cuda12.4
02aa490fc410:888827:889207 [0] NCCL INFO NCCL_IB_DISABLE set by environment to 1.
02aa490fc410:888827:889207 [0] NCCL INFO NET/Socket : Using [0]eth0:172.17.0.2<0>
02aa490fc410:888827:889207 [0] NCCL INFO Using non-device net plugin version 0
02aa490fc410:888827:889207 [0] NCCL INFO Using network Socket
02aa490fc410:888827:889210 [3] NCCL INFO Using non-device net plugin version 0
02aa490fc410:888827:889210 [3] NCCL INFO Using network Socket
02aa490fc410:888827:889209 [2]

Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.48.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


Epoch,Training Loss,Validation Loss
1,No log,0.056484


/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


KeyboardInterrupt: 

In [5]:
import numpy as np
from sklearn.metrics import f1_score

# Tokenize test dataset
test_dataset = tokenized_datasets["test"]

# Get predictions
predictions = trainer.predict(test_dataset)
preds = np.argmax(predictions.predictions, axis=-1)  # Get predicted labels
labels = test_dataset["label"]  # True labels

# Calculate F1 Macro score
f1_macro = f1_score(labels, preds, average="macro")
print(f"F1 Macro Score: {f1_macro:.4f}")


/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


F1 Macro Score: 0.0860


In [4]:
import torch
from sklearn.metrics import f1_score
from transformers import AutoTokenizer, AutoModelForSequenceClassification


# Load trained model and tokenizer
model_path = "models/"  # Path where the model was saved
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForSequenceClassification.from_pretrained(model_path)

# New data with simple sentences
new_data = [
    {"text": "I’m scared my credit score will never recover.", "label": "fear"},
    {"text": "It’s unfair how high the debt settlement fees are!", "label": "anger"},
    {"text": "My low credit score makes me feel hopeless.", "label": "sadness"},
    {"text": "I’m so happy my credit score is improving!", "label": "joy"},
    {"text": "What if my credit score drops even more?", "label": "fear"},
    {"text": "Why does debt settlement take so long?", "label": "anger"},
    {"text": "I feel defeated seeing my credit card debt.", "label": "sadness"},
    {"text": "It’s great that I finally paid off my debt!", "label": "joy"},
    {"text": "Lenders charging extra interest disgusts me.", "label": "anger"},
    {"text": "I’m terrified of my debt going to collections.", "label": "fear"},
    {"text": "Credit card fees are way too high!", "label": "anger"},
    {"text": "I feel lost trying to fix my bad credit.", "label": "sadness"},
    {"text": "I’m relieved my loan got approved!", "label": "joy"},
    {"text": "I hate that my interest rate keeps rising.", "label": "disgust"},
    {"text": "I am scared and What if my loan application gets rejected?", "label": "fear"},
    {"text": "Why does fixing credit take so long?", "label": "anger"}
]


# Split new data into texts and labels
texts = [item["text"] for item in new_data]
labels = [item["label"] for item in new_data]

# Tokenize unseen data
inputs = tokenizer(texts, padding=True, truncation=True, max_length=128, return_tensors="pt")
inputs = {key: value.to(model.device) for key, value in inputs.items()}  # Move inputs to correct device

# Perform inference
model.eval()  # Set model to evaluation mode
with torch.no_grad():
    outputs = model(**inputs)
    predictions = torch.argmax(outputs.logits, dim=1)  # Get label with highest probability

# Map predictions to label names
label_map = {i: label for i, label in enumerate(dataset["train"].features["label"].names)}  # Assuming label_map is defined
predicted_labels = [label_map[pred.item()] for pred in predictions]

print(predicted_labels)
# Calculate F1 score
f1 = f1_score(labels, predicted_labels, average="weighted")  # Use "weighted" for imbalanced classes
print(f"F1 Score: {f1:.4f}")

Some weights of T5ForSequenceClassification were not initialized from the model checkpoint at models/ and are newly initialized: ['classification_head.dense.bias', 'classification_head.dense.weight', 'classification_head.out_proj.bias', 'classification_head.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


['sadness', 'joy', 'sadness', 'joy', 'joy', 'sadness', 'sadness', 'joy', 'joy', 'sadness', 'sadness', 'sadness', 'sadness', 'sadness', 'sadness', 'sadness']
F1 Score: 0.1741
